### plot TIED results

In [ ]:
Expr <- t(apply(entire[,-c(17662:17668)], 2, function(x)x-(mean(x))))
Expr_ <- t(apply(TCGA_exp_pd[,-c(1:10)] ,2, function(x)x-(mean(x))))
write.table(Expr,"entire.txt",sep = "\t")
write.table(Expr_,"TCGA.txt",sep = "\t")
entire_TIDE = read.csv("entire__tide.csv",row.names = 1)[c(10,11)][rownames(annotation_col_),]
TCGA_TIDE$Risk = annotation_col_F5C_TCGA$Risk
TCGA_TIDE = gather(TCGA_TIDE,set,value,-Risk)
mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
g3 = ggplot(TCGA_TIDE,aes(set,value,fill = Risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "") +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=90,vjust = 0.7, hjust = 1))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
  stat_compare_means(aes(group = Risk,label = ..p.signif..),method = "wilcox.test")
g3
ggsave("TIED.pdf",g3,width = 5,height =5)
save(TCGA_TIDE,file = "TCGA_TIDE.rdata")

### caculate IPS

In [ ]:
library(IOBR)
load("OKModel_risk.rdata")
library(IOBR)
library(purrr) 
library(RColorBrewer)
library(tidyr)
library(ggpubr)
train_risk_= entire_risk[rownames(train_risk),]
fpkm_in = t(TCGA_risk[,11:17671])
fpkm_in = t(entire_risk[,1:17661])
fpkm_in =t(train_risk_[,1:17661])
ips_tcga<-deconvo_tme(eset = fpkm_in, method = "ips", plot= F)
estimate = deconvo_tme(eset = fpkm_in, method = "estimate", plot= T)
group = data.frame(ID = rownames(entire_risk),risk = entire_risk[,"risk"])
ips_ = merge(group,ips,by = "ID")
group_tcga = data.frame(ID = rownames(TCGA_risk),risk = TCGA_risk[,"risk"])
ips_tcga_ =  merge(group_tcga,ips_tcga,by = "ID")
estimate_ = merge(group,estimate,by = "ID")
ips_tcga__ = ips_tcga_[order(ips_tcga_$MHC_IPS,decreasing = T),][-c(1:4,6,16,11),]
ips__ =ips_[order(ips_$MHC_IPS,decreasing = T),][-c(1:4,6,16,11),]
pp = map(c(3,6,7,8),function(i){
ggplot(ips__,aes(ips__[,2],ips__[,i],fill = risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = paste0(colnames(ips__)[i])) +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=0,vjust = 0, hjust = 0.5))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
stat_compare_means(comparisons = my_comparisons)+theme(text = element_text(size=25))
})
g2 = ggplot(ips__,aes(ips__[,2],ips__[,3],fill = risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "MHC_IPS") +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=0,vjust = 0, hjust = 0.5))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
stat_compare_means(comparisons = my_comparisons)+theme(text = element_text(size=25))
pdf("IOBR_entire.pdf",width = 15,height = 5)
p = cowplot::plot_grid(plotlist = pp,align = "h",nrow  = 1)
p
dev.off()
my_comparisons = list(c("high","low")) 
g3 = ggplot(ips__,aes(risk,IPS_IPS,fill = risk)) + 
  geom_violin(color = "black") + 
geom_boxplot(outlier.shape = 21,fill = "white",width =.04) +
  stat_summary(fun=mean, geom="point", shape=23, size=3, fill="black", position=position_dodge(0.75))+
  theme_bw() + 
  labs(x = "", y = "IPS") +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=0,vjust = 0, hjust = 0.5))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
stat_compare_means(comparisons = my_comparisons)+
theme(text = element_text(size = 25))
my_comparisons = list(c("",""))
mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
g3 = ggplot(ips_,aes(set,value,fill = risk)) + 
  geom_boxplot(outlier.shape = 21,color = "black") + 
  theme_bw() + 
  labs(x = "", y = "Gene expression") +
  theme(legend.position = "top") + 
  theme(axis.text.x = element_text(angle=0,vjust = 0, hjust = 0.5))+
  scale_fill_manual(values = mypalette(22)[c(1,6)])+ 
  stat_compare_means(aes(group = risk,label = ..p.signif..),method = "wilcox.test")#添加的是星号

### Analyzing drugs sensitivity

In [ ]:
library(data.table)
library(pRRophetic)
library(tidyr)
load("OKModel_risk.rdata")
exprDataBortezomib = exprDataBortezomib[!is.na(rownames(exprDataBortezomib)),]
testMatrix = as.matrix(t(entire_risk[,1:17661]))
predictedPtype <- pRRopheticPredict(testMatrix=testMatrix, 
                                    drug="Cisplatin",
                                    tissueType = "allSolidTumors", 
                                    batchCorrect = "eb",
                                    selection=1,
                                    dataset = "cgp2014")
drugs_risk =df ;save(drugs_risk,drugs,file = "drugs_risk.rdata")
library(purrr)
library(RColorBrewer)
library(tidyr)
library(ggpubr)
pp <- map(3:14, function(i) {
  my_comparisons <- list(c("high", "low"))
  mypalette <- colorRampPalette(brewer.pal(8, "Set1"))

  ggplot(drugs_risk, aes(risk, drugs_risk[, i], fill = risk)) +
    geom_boxplot(outlier.shape = 21, color = "black") +
    theme_bw() +
    labs(x = "", y = paste0(colnames(drugs_risk)[i], " senstivity(IC50)"),size = 15) +
    theme(legend.position = "top",axis.text.x = element_text(size = 15),axis.text.y = element_text(size = 15)) +
    scale_fill_manual(values = mypalette(22)[c(1, 6)]) +
    stat_compare_means(comparisons = my_comparisons,size = 2.5) +
    ggtitle(paste0(colnames(drugs_risk)[i]))+
    theme(plot.title = element_text(hjust = 0.5, vjust = 1.5, size = 15)) +
    scale_x_discrete(labels = c("High","Low"))+
    guides(fill = "none")
})
library(cowplot)
x1 = plot_grid(plotlist = pp,ncol = 6)
ggsave("test.pdf",x1,width = 16,height = 7)